In [1]:
import sage.libs.lrcalc.lrcalc as lrcalc
import math
import numpy as np
from itertools import zip_longest

In [2]:
indexDict = []
ind = -1

In [3]:
def lamsComp(lams,a,b):
    
    #     :param lams: an array of partitions
    #     :param    a: number of rows
    #     :param    b: number of columns
    #     :return    : complement of lams under rectangle a*b

    result = []
    
    for lam in lams:
        comp = []
        for i in range(a):
            if i >= len(lam):
                comp = [b]*(a-i) + comp
                break
            if (b-lam[i]) == 0: continue
            comp.insert(0,b-lam[i])
            
        result.append(comp)
    
    return result

In [61]:
def schurVec(l1,l2):
    #given two partition, output the LR vector wrt the index Dict
    global indexDict,ind
    result = [0]*50000 #should find a better way to do this ..
    
    prod = lrcalc.mult(l1,l2)
    count = 0
    for p in prod:
        if p not in indexDict:
#             print("!!!!")
            indexDict.append(p)
        current = indexDict.index(p)
        result[current] = prod[p]
    
    return result

In [5]:
def schurVecBefore(l1,l2):
    #given two partition, output the LR vector wrt the index Dict
    global indexDict,ind
    result = [0]*10000 #should find a better way to do this ..
    
    prod = lrcalc.mult(l1,l2)
    count = 0
    for p in prod:
        current = indexDict.get(p)
        if current == None:
            ind += 1
            count += 1
            indexDict[p] = ind
            current = ind
#         print(current,p)
        result[current] = prod[p]
#     print(count)
#     if count == 0:
#         print(l1,l2)
    
    return result

In [6]:
def subsetPartRec(a,b):
    #return a list of subset partitions of a rectangle a*b
    result = []
    if (a*b) % 2 == 1:
        for i in range(math.floor((a*b)/2)+1):
            result += Partitions(i, max_part=b, max_length=a).list()
    else:
        for i in range(math.floor((a*b)/2)):
            result += Partitions(i, max_part=b, max_length=a).list()
            
        comps = []
        for p in Partitions((math.floor((a*b)/2)), max_part=b, max_length=a).list():
            comp = lamsComp([p],a,b)[0]
            if p not in comps:
                comps.append(comp)
                result.append(p)
            
    return result

In [7]:
def plusOne(lam,a):
    #partition plus 1^a
    result = [i+1 for i in lam]
    if len(result) == a: return result
    result += [1]*(a-len(result))
    return result

In [8]:
def findMin(vectors):
    # dominate order
    for v in vectors:
        flag = 0
        for u in vectors:
            if v != u:
                if Partition(v).dominates(Partition(u)) or not Partition(u).dominates(Partition(v)):
#                     print(v,u)
                    flag = 1
                    break
        if flag == 0:
            print(v)

In [9]:
def lexMin(l1,l2):
    testList = lrcalc.mult(l1,l2)
    m = [k for k,v in testList.items() if v == 1][0]

    for g in testList:
        current = [x-y for x,y in zip_longest(g, m, fillvalue=0)]
        for e in current:
            if e < 0:
                m = g
                break
            elif e > 0:
                break
    return m

In [10]:
def lexMax(l1,l2):
    testList = lrcalc.mult(l1,l2)
    m = [k for k,v in testList.items() if v == 1][0]

    for g in testList:
        current = [x-y for x,y in zip_longest(m, g, fillvalue=0)]
        for e in current:
            if e == 0: continue
            if e < 0:
                m = g
                break
            if e > 0:
                break
    return m

In [64]:
def lexOrder(l1,l2):
    # if l1 < l2, return negative
    for i,e in enumerate(l1):
        if e == l2[i]:
            continue
        if e < l2[i]:
            return -1
        if e > l2[i]:
            return 1
    return 0

In [73]:

for a in range(2,3):
    for b in range(2,3):
        
        a = 3
        b = 3
        print(a,b)
        
        lams = subsetPartRec(a,b)
        lamsC = lamsComp(lams,a,b)
        
        print(lams)
        print(lamsC)
        indexDict = []
#         33
#         indexDict = [[3, 3, 3], [3, 3, 1], [3, 2, 2], [4, 3, 2], [3, 3, 2, 1], [3, 2], [3, 1, 1], [4, 3], [4, 2, 1], [3, 2, 1, 1], [5, 3, 1], [4, 3, 1, 1], [2, 2, 1], [2, 2, 2, 1], [4, 2, 2, 1], [3, 2, 2, 1, 1], [3], [4, 1], [5, 2], [6, 3], [2, 1], [1, 1, 1], [2, 1, 1, 1], [5, 1, 1], [4, 1, 1, 1], [3, 1, 1, 1, 1], [2, 2, 1, 1, 1], [5, 2, 2], [5, 2, 1, 1], [4, 4, 1], [4, 2, 1, 1, 1], [3, 3, 1, 1, 1], [3, 2, 2, 2], [2, 2, 2, 1, 1, 1], [1], [5], [6, 1], [6, 2, 1], [5, 4], [1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1], [3, 2, 1, 1, 1, 1], [2, 2, 2, 2, 1]]
#         22
#         indexDict = [[2, 2], [2], [1, 1], [3, 1], [2, 1, 1], [], [4], [1, 1, 1, 1]]
        ind = 0

        vectors = []

        
        for i in range(len(lams)):
            v = schurVec(lams[i],lamsC[i])
            vectors.append(v)

        A = span(vectors,ZZ)
        print(A.dimension() == len(vectors))
        
        maxInd = np.max(np.nonzero(vectors[-1]))
        vectors = np.array(vectors)
        vectors = vectors[:,0:int(maxInd)+1]

        for v in vectors:
            print(v)
        vectors = [list(v) for v in vectors]
        
        n = len(vectors)
        m = len(vectors[0])
        countDel = 0
        
        for i,e in enumerate(indexDict):
            print(i+1, e)
            
        while countDel < n-1:
            i = n - 1
            while i > 0:
    #         for i in reversed(range(1,len(vectors))):
                v = vectors[i]
                if v[-1] == -1:
                    i -= 1
                    continue
                mark = 0
                for j in reversed(range(len(v))):
                    if v[j] == 1:
                        current = vectors[:i] + vectors[i+1:]
                        flag = 0
                        for u in current:
                            if u[j] == 1:
                                flag = 1
                                break
                        if flag == 0:
                            mark = 1
#                             print(lams[i], i, v, j, indexDict[j])
                            temp = [0]*(m-1) + [-1]
                            vectors = vectors[:i] + [temp] + vectors[i+1:]
                            countDel += 1
#                             i = len(vectors) - 1
    #                         print(np.array(vectors))
                            print(lams[i], indexDict[j])
                            break
                if mark == 0:
                    i = i - 1
                else:
                    break
        

3 3
[[], [1], [2], [1, 1], [3], [2, 1], [1, 1, 1], [3, 1], [2, 2], [2, 1, 1]]
[[3, 3, 3], [3, 3, 2], [3, 3, 1], [3, 2, 2], [3, 3], [3, 2, 1], [2, 2, 2], [3, 2], [3, 1, 1], [2, 2, 1]]
True
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[1 2 2 2 1 2 1 0 1 1 1 1 1 1 0 0 0 0 0]
[1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0]
[1 1 2 1 2 1 0 1 0 0 1 0 1 1 0 1 1 0 0]
[1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0]
[1 2 1 1 0 1 2 0 1 1 0 1 0 0 1 0 0 1 1]
1 [3, 3, 3]
2 [3, 3, 2, 1]
3 [4, 3, 2]
4 [4, 3, 1, 1]
5 [5, 3, 1]
6 [4, 2, 2, 1]
7 [3, 2, 2, 1, 1]
8 [6, 3]
9 [3, 2, 2, 2]
10 [4, 2, 1, 1, 1]
11 [5, 2, 1, 1]
12 [3, 3, 1, 1, 1]
13 [4, 4, 1]
14 [5, 2, 2]
15 [2, 2, 2, 1, 1, 1]
16 [5, 4]
17 [6, 2, 1]
18 [2, 2, 2, 2, 1]
19 [3, 2, 1, 1, 1, 1]
[2, 1, 1] [3, 2, 1, 1, 1, 1]
[3, 1] [6, 2, 1]
[1, 1, 1] [2, 2, 2, 1, 1, 1]
[2, 1] [5, 2, 2]
[2, 2] [5, 2, 1, 1]
[3] [6, 3]


In [68]:
lrcalc.mult([1],[2,1])

{[2, 2]: 1, [3, 1]: 1, [2, 1, 1]: 1}

In [47]:
test = [[1,2,3],[4,5,6],[7,8,9]]
test = np.array(test)
test = list(test)
test = test[:1]+test[2:]
test

[array([1, 2, 3]), array([7, 8, 9])]

In [10]:
for a in range(2,7):
    for b in range(2,7):
        print(a,b)
        lams = subsetPartRec(a,b)
        lamsC = lamsComp(lams,a,b)

        indexDict = {}
        ind = 0
        
        for i,u in enumerate(lams):
            uC = lamsC[i]
            uT = Partition(u).conjugate()
            right = [x+y for x,y in zip_longest(uT, Partition(uC).conjugate(), fillvalue=0)]
            
            for j,l in enumerate(lams):
                lC = lamsC[j]
                lT = Partition(l).conjugate()
                left = [x+y for x,y in zip_longest(lT, Partition(lC).conjugate(), fillvalue=0)]
                
                if Partition(right).dominates(left):
                    implyRight = [x+y for x,y in zip_longest(u, uC, fillvalue=0)]
                    implyLeft = [x+y for x,y in zip_longest(l, lC, fillvalue=0)]
                    if not Partition(implyRight).dominates(implyLeft):
                        print("!!!!")
                        print(u, uC, uT, l, lC, lT)
                        print(right, left, implyRight, implyLeft)
         

2 2
!!!!
[] [2, 2] [] [2] [2] [1, 1]
[2, 2] [2, 2] [2, 2] [4]
!!!!
[1] [2, 1] [1] [2] [2] [1, 1]
[3, 1] [2, 2] [3, 1] [4]
!!!!
[1, 1] [1, 1] [2] [1] [2, 1] [1]
[4] [3, 1] [2, 2] [3, 1]
!!!!
[1, 1] [1, 1] [2] [2] [2] [1, 1]
[4] [2, 2] [2, 2] [4]
2 3
!!!!
[] [3, 3] [] [3] [3] [1, 1, 1]
[2, 2, 2] [2, 2, 2] [3, 3] [6]
!!!!
[1] [3, 2] [1] [2] [3, 1] [1, 1]
[3, 2, 1] [3, 2, 1] [4, 2] [5, 1]
!!!!
[1] [3, 2] [1] [3] [3] [1, 1, 1]
[3, 2, 1] [2, 2, 2] [4, 2] [6]
!!!!
[2] [3, 1] [1, 1] [3] [3] [1, 1, 1]
[3, 2, 1] [2, 2, 2] [5, 1] [6]
!!!!
[1, 1] [2, 2] [2] [1] [3, 2] [1]
[4, 2] [3, 2, 1] [3, 3] [4, 2]
!!!!
[1, 1] [2, 2] [2] [2] [3, 1] [1, 1]
[4, 2] [3, 2, 1] [3, 3] [5, 1]
!!!!
[1, 1] [2, 2] [2] [3] [3] [1, 1, 1]
[4, 2] [2, 2, 2] [3, 3] [6]
!!!!
[1, 1] [2, 2] [2] [2, 1] [2, 1] [2, 1]
[4, 2] [4, 2] [3, 3] [4, 2]
!!!!
[2, 1] [2, 1] [2, 1] [2] [3, 1] [1, 1]
[4, 2] [3, 2, 1] [4, 2] [5, 1]
!!!!
[2, 1] [2, 1] [2, 1] [3] [3] [1, 1, 1]
[4, 2] [2, 2, 2] [4, 2] [6]
2 4
!!!!
[] [4, 4] [] [4] [4] [1, 1, 1, 1]

[3, 3, 3] [3, 3, 3] [3, 3, 3] [6, 3]
!!!!
[1] [3, 3, 2] [1] [2] [3, 3, 1] [1, 1]
[4, 3, 2] [4, 3, 2] [4, 3, 2] [5, 3, 1]
!!!!
[1] [3, 3, 2] [1] [3] [3, 3] [1, 1, 1]
[4, 3, 2] [3, 3, 3] [4, 3, 2] [6, 3]
!!!!
[1] [3, 3, 2] [1] [3, 1] [3, 2] [2, 1, 1]
[4, 3, 2] [4, 3, 2] [4, 3, 2] [6, 3]
!!!!
[2] [3, 3, 1] [1, 1] [3] [3, 3] [1, 1, 1]
[4, 3, 2] [3, 3, 3] [5, 3, 1] [6, 3]
!!!!
[2] [3, 3, 1] [1, 1] [3, 1] [3, 2] [2, 1, 1]
[4, 3, 2] [4, 3, 2] [5, 3, 1] [6, 3]
!!!!
[1, 1] [3, 2, 2] [2] [2] [3, 3, 1] [1, 1]
[5, 3, 1] [4, 3, 2] [4, 3, 2] [5, 3, 1]
!!!!
[1, 1] [3, 2, 2] [2] [3] [3, 3] [1, 1, 1]
[5, 3, 1] [3, 3, 3] [4, 3, 2] [6, 3]
!!!!
[1, 1] [3, 2, 2] [2] [2, 1] [3, 2, 1] [2, 1]
[5, 3, 1] [5, 3, 1] [4, 3, 2] [5, 3, 1]
!!!!
[1, 1] [3, 2, 2] [2] [3, 1] [3, 2] [2, 1, 1]
[5, 3, 1] [4, 3, 2] [4, 3, 2] [6, 3]
!!!!
[1, 1] [3, 2, 2] [2] [2, 2] [3, 1, 1] [2, 2]
[5, 3, 1] [5, 3, 1] [4, 3, 2] [5, 3, 1]
!!!!
[2, 1] [3, 2, 1] [2, 1] [3] [3, 3] [1, 1, 1]
[5, 3, 1] [3, 3, 3] [5, 3, 1] [6, 3]
!!!!
[2, 1] [3, 2,

[1, 1] [5, 4, 4] [2] [4, 1] [5, 4, 1] [2, 1, 1, 1]
[5, 3, 3, 3, 1] [5, 3, 3, 3, 1] [6, 5, 4] [9, 5, 1]
!!!!
[1, 1] [5, 4, 4] [2] [5, 1] [5, 4] [2, 1, 1, 1, 1]
[5, 3, 3, 3, 1] [4, 3, 3, 3, 2] [6, 5, 4] [10, 5]
!!!!
[1, 1] [5, 4, 4] [2] [5, 2] [5, 3] [2, 2, 1, 1, 1]
[5, 3, 3, 3, 1] [4, 4, 3, 2, 2] [6, 5, 4] [10, 5]
!!!!
[1, 1] [5, 4, 4] [2] [5, 1, 1] [4, 4] [3, 1, 1, 1, 1]
[5, 3, 3, 3, 1] [5, 3, 3, 3, 1] [6, 5, 4] [9, 5, 1]
!!!!
[3] [5, 5, 2] [1, 1, 1] [4] [5, 5, 1] [1, 1, 1, 1]
[4, 4, 3, 2, 2] [4, 3, 3, 3, 2] [8, 5, 2] [9, 5, 1]
!!!!
[3] [5, 5, 2] [1, 1, 1] [5] [5, 5] [1, 1, 1, 1, 1]
[4, 4, 3, 2, 2] [3, 3, 3, 3, 3] [8, 5, 2] [10, 5]
!!!!
[3] [5, 5, 2] [1, 1, 1] [5, 1] [5, 4] [2, 1, 1, 1, 1]
[4, 4, 3, 2, 2] [4, 3, 3, 3, 2] [8, 5, 2] [10, 5]
!!!!
[3] [5, 5, 2] [1, 1, 1] [5, 2] [5, 3] [2, 2, 1, 1, 1]
[4, 4, 3, 2, 2] [4, 4, 3, 2, 2] [8, 5, 2] [10, 5]
!!!!
[2, 1] [5, 4, 3] [2, 1] [3] [5, 5, 2] [1, 1, 1]
[5, 4, 3, 2, 1] [4, 4, 3, 2, 2] [7, 5, 3] [8, 5, 2]
!!!!
[2, 1] [5, 4, 3] [2, 1] [4] [5, 

[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [4] [5, 5, 1] [1, 1, 1, 1]
[6, 4, 3, 2] [4, 3, 3, 3, 2] [8, 5, 2] [9, 5, 1]
!!!!
[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [5] [5, 5] [1, 1, 1, 1, 1]
[6, 4, 3, 2] [3, 3, 3, 3, 3] [8, 5, 2] [10, 5]
!!!!
[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [4, 1] [5, 4, 1] [2, 1, 1, 1]
[6, 4, 3, 2] [5, 3, 3, 3, 1] [8, 5, 2] [9, 5, 1]
!!!!
[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [5, 1] [5, 4] [2, 1, 1, 1, 1]
[6, 4, 3, 2] [4, 3, 3, 3, 2] [8, 5, 2] [10, 5]
!!!!
[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [4, 2] [5, 3, 1] [2, 2, 1, 1]
[6, 4, 3, 2] [5, 4, 3, 2, 1] [8, 5, 2] [9, 5, 1]
!!!!
[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [5, 2] [5, 3] [2, 2, 1, 1, 1]
[6, 4, 3, 2] [4, 4, 3, 2, 2] [8, 5, 2] [10, 5]
!!!!
[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [5, 1, 1] [4, 4] [3, 1, 1, 1, 1]
[6, 4, 3, 2] [5, 3, 3, 3, 1] [8, 5, 2] [9, 5, 1]
!!!!
[4, 2, 1] [4, 3, 1] [3, 2, 1, 1] [4, 3] [5, 2, 1] [2, 2, 2, 1]
[6, 4, 3, 2] [5, 4, 3, 2, 1] [8, 5, 2] [9, 5, 1]
!!!!
[3, 3, 1] [4, 2, 2] [3, 2, 2] [3] [5, 5, 2] [1, 1, 1]
[6, 5, 3, 1] [4, 4,

[6, 4, 4, 2, 2] [5, 3, 3, 3, 3, 1] [8, 6, 4] [11, 6, 1]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [4, 2] [6, 4, 2] [2, 2, 1, 1]
[6, 4, 4, 2, 2] [5, 5, 3, 3, 1, 1] [8, 6, 4] [10, 6, 2]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [4, 1, 1] [5, 5, 2] [3, 1, 1, 1]
[6, 4, 4, 2, 2] [6, 4, 3, 3, 2] [8, 6, 4] [9, 6, 3]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [6, 1] [6, 5] [2, 1, 1, 1, 1, 1]
[6, 4, 4, 2, 2] [4, 3, 3, 3, 3, 2] [8, 6, 4] [12, 6]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [5, 2] [6, 4, 1] [2, 2, 1, 1, 1]
[6, 4, 4, 2, 2] [5, 4, 3, 3, 2, 1] [8, 6, 4] [11, 6, 1]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [5, 1, 1] [5, 5, 1] [3, 1, 1, 1, 1]
[6, 4, 4, 2, 2] [6, 3, 3, 3, 3] [8, 6, 4] [10, 6, 2]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [4, 3] [6, 3, 2] [2, 2, 2, 1]
[6, 4, 4, 2, 2] [5, 5, 4, 2, 1, 1] [8, 6, 4] [10, 6, 2]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [6, 2] [6, 4] [2, 2, 1, 1, 1, 1]
[6, 4, 4, 2, 2] [4, 4, 3, 3, 2, 2] [8, 6, 4] [12, 6]
!!!!
[3, 1, 1] [5, 5, 3] [3, 1, 1] [6, 1, 1] [5, 5] [3, 1, 1, 1, 1, 1]
[6, 4, 4, 2, 2] [5, 3

...

[8, 6, 5, 3, 2] [7, 4, 4, 4, 4, 1] [8, 7, 5, 4] [11, 6, 6, 1]
!!!!
[3, 2, 1, 1] [5, 5, 4, 3] [4, 2, 1] [4, 3] [6, 6, 3, 2] [2, 2, 2, 1]
[8, 6, 5, 3, 2] [6, 6, 5, 3, 2, 2] [8, 7, 5, 4] [10, 9, 3, 2]
!!!!
[3, 2, 1, 1] [5, 5, 4, 3] [4, 2, 1] [4, 2, 1] [6, 5, 4, 2] [3, 2, 1, 1]
[8, 6, 5, 3, 2] [7, 6, 4, 4, 2, 1] [8, 7, 5, 4] [10, 7, 5, 2]
!!!!
[3, 2, 1, 1] [5, 5, 4, 3] [4, 2, 1] [4, 1, 1, 1] [5, 5, 5, 2] [4, 1, 1, 1]
[8, 6, 5, 3, 2] [8, 5, 4, 4, 3] [8, 7, 5, 4] [9, 6, 6, 3]
!!!!
[3, 2, 1, 1] [5, 5, 4, 3] [4, 2, 1] [3, 3, 1] [6, 5, 3, 3] [3, 2, 2]
[8, 6, 5, 3, 2] [7, 6, 6, 2, 2, 1] [8, 7, 5, 4] [9, 8, 4, 3]
!!!!
[3, 2, 1, 1] [5, 5, 4, 3] [4, 2, 1] [3, 2, 2] [6, 4, 4, 3] [3, 3, 1]
[8, 6, 5, 3, 2] [7, 7, 5, 3, 1, 1] [8, 7, 5, 4] [9, 6, 6, 3]
!!!!
[3, 2, 1, 1] [5, 5, 4, 3] [4, 2, 1] [6, 2] [6, 6, 4] [2, 2, 1, 1, 1, 1]
[8, 6, 5, 3, 2] [5, 5, 4, 4, 3, 3] [8, 7, 5, 4] [12, 8, 4]
!!!!
[3, 2, 1, 1] [5, 5, 4, 3] [4, 2, 1] [6, 1, 1] [6, 5, 5] [3, 1, 1, 1, 1, 1]
[8, 6, 5, 3, 2] [6, 4, 4, 4, 4, 2] [8, 

[5, 1, 1, 1] [5, 5, 5, 1] [4, 1, 1, 1, 1] [5, 5] [6, 6, 1, 1] [2, 2, 2, 2, 2]
[8, 4, 4, 4, 4] [6, 4, 4, 4, 4, 2] [10, 6, 6, 2] [11, 11, 1, 1]
!!!!
[5, 1, 1, 1] [5, 5, 5, 1] [4, 1, 1, 1, 1] [5, 4, 1] [6, 5, 2, 1] [3, 2, 2, 2, 1]
[8, 4, 4, 4, 4] [7, 5, 4, 4, 3, 1] [10, 6, 6, 2] [11, 9, 3, 1]
!!!!
[5, 1, 1, 1] [5, 5, 5, 1] [4, 1, 1, 1, 1] [6, 5] [6, 6, 1] [2, 2, 2, 2, 2, 1]
[8, 4, 4, 4, 4] [5, 4, 4, 4, 4, 3] [10, 6, 6, 2] [12, 11, 1]
!!!!
[5, 1, 1, 1] [5, 5, 5, 1] [4, 1, 1, 1, 1] [6, 4, 1] [6, 5, 2] [3, 2, 2, 2, 1, 1]
[8, 4, 4, 4, 4] [6, 5, 4, 4, 3, 2] [10, 6, 6, 2] [12, 9, 3]
!!!!
[5, 1, 1, 1] [5, 5, 5, 1] [4, 1, 1, 1, 1] [5, 5, 1] [6, 5, 1, 1] [3, 2, 2, 2, 2]
[8, 4, 4, 4, 4] [7, 4, 4, 4, 4, 1] [10, 6, 6, 2] [11, 10, 2, 1]
!!!!
[5, 1, 1, 1] [5, 5, 5, 1] [4, 1, 1, 1, 1] [6, 6] [6, 6] [2, 2, 2, 2, 2, 2]
[8, 4, 4, 4, 4] [4, 4, 4, 4, 4, 4] [10, 6, 6, 2] [12, 12]
!!!!
[5, 1, 1, 1] [5, 5, 5, 1] [4, 1, 1, 1, 1] [6, 5, 1] [6, 5, 1] [3, 2, 2, 2, 2, 1]
[8, 4, 4, 4, 4] [6, 4, 4, 4, 4, 2] [10, 6, 6,

[3, 3, 2] [6, 4, 3, 3] [3, 3, 2] [6, 1, 1, 1] [5, 5, 5] [4, 1, 1, 1, 1, 1]
[7, 7, 6, 2, 1, 1] [7, 4, 4, 4, 4, 1] [9, 7, 5, 3] [11, 6, 6, 1]
!!!!
[3, 3, 2] [6, 4, 3, 3] [3, 3, 2] [5, 4] [6, 6, 2, 1] [2, 2, 2, 2, 1]
[7, 7, 6, 2, 1, 1] [6, 5, 4, 4, 3, 2] [9, 7, 5, 3] [11, 10, 2, 1]
!!!!
[3, 3, 2] [6, 4, 3, 3] [3, 3, 2] [5, 3, 1] [6, 5, 3, 1] [3, 2, 2, 1, 1]
[7, 7, 6, 2, 1, 1] [7, 5, 5, 3, 3, 1] [9, 7, 5, 3] [11, 8, 4, 1]
!!!!
[3, 3, 2] [6, 4, 3, 3] [3, 3, 2] [5, 2, 2] [6, 4, 4, 1] [3, 3, 1, 1, 1]
[7, 7, 6, 2, 1, 1] [7, 6, 4, 4, 2, 1] [9, 7, 5, 3] [11, 6, 6, 1]
!!!!
[3, 3, 2] [6, 4, 3, 3] [3, 3, 2] [4, 4, 1] [6, 5, 2, 2] [3, 2, 2, 2]
[7, 7, 6, 2, 1, 1] [7, 6, 4, 4, 2, 1] [9, 7, 5, 3] [10, 9, 3, 2]
!!!!
[3, 3, 2] [6, 4, 3, 3] [3, 3, 2] [4, 3, 2] [6, 4, 3, 2] [3, 3, 2, 1]
[7, 7, 6, 2, 1, 1] [7, 7, 5, 3, 1, 1] [9, 7, 5, 3] [10, 7, 5, 2]
!!!!
[3, 3, 2] [6, 4, 3, 3] [3, 3, 2] [6, 4] [6, 6, 2] [2, 2, 2, 2, 1, 1]
[7, 7, 6, 2, 1, 1] [5, 5, 4, 4, 3, 3] [9, 7, 5, 3] [12, 10, 2]
!!!!
[3, 3, 2] [6, 4,

[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [2, 2, 2] [6, 4, 4, 4] [3, 3]
[8, 8, 4, 4] [7, 7, 4, 4, 1, 1] [6, 6, 6, 6] [8, 6, 6, 4]
!!!!
[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [2, 2, 1, 1] [5, 5, 4, 4] [4, 2]
[8, 8, 4, 4] [8, 6, 4, 4, 2] [6, 6, 6, 6] [7, 7, 5, 5]
!!!!
[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [6, 1] [6, 6, 5] [2, 1, 1, 1, 1, 1]
[8, 8, 4, 4] [5, 4, 4, 4, 4, 3] [6, 6, 6, 6] [12, 7, 5]
!!!!
[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [5, 2] [6, 6, 4, 1] [2, 2, 1, 1, 1]
[8, 8, 4, 4] [6, 5, 4, 4, 3, 2] [6, 6, 6, 6] [11, 8, 4, 1]
!!!!
[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [5, 1, 1] [6, 5, 5, 1] [3, 1, 1, 1, 1]
[8, 8, 4, 4] [7, 4, 4, 4, 4, 1] [6, 6, 6, 6] [11, 6, 6, 1]
!!!!
[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [4, 3] [6, 6, 3, 2] [2, 2, 2, 1]
[8, 8, 4, 4] [6, 6, 5, 3, 2, 2] [6, 6, 6, 6] [10, 9, 3, 2]
!!!!
[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [4, 2, 1] [6, 5, 4, 2] [3, 2, 1, 1]
[8, 8, 4, 4] [7, 6, 4, 4, 2, 1] [6, 6, 6, 6] [10, 7, 5, 2]
!!!!
[2, 2, 2, 2] [4, 4, 4, 4] [4, 4] [4, 1, 1, 1] [5, 5, 5, 2] [4, 1, 1, 1]
[8, 8, 4, 4] [

KeyboardInterrupt: 

In [38]:
for a in range(6,7):
    for b in range(6,7):
        print(a,b)
        lams = subsetPartRec(a,b)
        lamsC = lamsComp(lams,a,b)

        indexDict = {}
        ind = 0

        pairs = []
        rep = []

        for i in range(len(lams)):
            partMax = [x+y for x,y in zip_longest(lams[i], lamsC[i], fillvalue=0)]
            partMin = Partition([x+y for x,y in zip_longest(Partition(lams[i]).conjugate(), Partition(lamsC[i]).conjugate(), fillvalue=0)]).conjugate()
            current = [partMin, partMax]
#             current = [lexMin(lams[i],lamsC[i]), lexMax(lams[i], lamsC[i])]
            if current in pairs:
#             if current == [[3, 2, 2, 1, 1], [5, 3, 1]]:
#                 print(lams[i],lamsC[i],current)
                rep.append(current)
            pairs.append(current)
        
        for r in rep:
            print("=====")
            print(r)
            for i in range(len(lams)):
                partMax = [x+y for x,y in zip_longest(lams[i], lamsC[i], fillvalue=0)]
                partMin = Partition([x+y for x,y in zip_longest(Partition(lams[i]).conjugate(), Partition(lamsC[i]).conjugate(), fillvalue=0)]).conjugate()
                current = [partMin, partMax]
                if current == r:
    #                 print(current)
                    print(lams[i],lamsC[i])
        

6 6
=====
[[6, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1], [11, 6, 6, 6, 6, 1]]
[5, 1, 1, 1, 1] [6, 5, 5, 5, 5, 1]
[6, 1, 1, 1, 1, 1] [5, 5, 5, 5, 5]
=====
[[6, 5, 5, 5, 5, 4, 2, 1, 1, 1, 1], [11, 7, 6, 6, 5, 1]]
[5, 2, 1, 1, 1] [6, 5, 5, 5, 4, 1]
[6, 2, 1, 1, 1, 1] [5, 5, 5, 5, 4]
=====
[[6, 5, 5, 5, 5, 3, 3, 1, 1, 1, 1], [11, 8, 6, 6, 4, 1]]
[5, 3, 1, 1, 1] [6, 5, 5, 5, 3, 1]
[6, 3, 1, 1, 1, 1] [5, 5, 5, 5, 3]
=====
[[6, 5, 5, 5, 4, 4, 2, 2, 1, 1, 1], [11, 7, 7, 5, 5, 1]]
[5, 2, 2, 1, 1] [6, 5, 5, 4, 4, 1]
[6, 2, 2, 1, 1, 1] [5, 5, 5, 4, 4]
=====
[[6, 5, 5, 5, 5, 4, 2, 1, 1, 1, 1], [11, 9, 6, 6, 3, 1]]
[5, 4, 1, 1, 1] [6, 5, 5, 5, 2, 1]
[6, 4, 1, 1, 1, 1] [5, 5, 5, 5, 2]
=====
[[6, 5, 5, 5, 4, 3, 3, 2, 1, 1, 1], [11, 8, 7, 5, 4, 1]]
[5, 3, 2, 1, 1] [6, 5, 5, 4, 3, 1]
[6, 3, 2, 1, 1, 1] [5, 5, 5, 4, 3]
=====
[[6, 5, 5, 4, 4, 4, 2, 2, 2, 1, 1], [11, 7, 6, 6, 5, 1]]
[5, 2, 2, 2, 1] [6, 5, 4, 4, 4, 1]
[6, 2, 2, 2, 1, 1] [5, 5, 4, 4, 4]
=====
[[6, 6, 6, 5, 5, 5, 1, 1, 1], [12, 11, 6, 6, 1]]
[6, 5, 1, 

[5, 4, 2, 2, 1] [6, 5, 4, 4, 2, 1]
[6, 4, 2, 2, 1, 1] [5, 5, 4, 4, 2]
[5, 5, 2, 2, 2] [6, 4, 4, 4, 1, 1]
[6, 5, 2, 2, 2, 1] [5, 4, 4, 4, 1]
=====
[[6, 5, 5, 4, 4, 4, 2, 2, 2, 1, 1], [11, 9, 8, 4, 3, 1]]
[5, 4, 4, 2, 1] [6, 5, 4, 2, 2, 1]
[6, 4, 4, 2, 1, 1] [5, 5, 4, 2, 2]
=====
[[6, 5, 5, 4, 3, 3, 3, 3, 2, 1, 1], [11, 9, 6, 6, 3, 1]]
[5, 4, 3, 3, 1] [6, 5, 3, 3, 2, 1]
[6, 4, 3, 3, 1, 1] [5, 5, 3, 3, 2]
=====
[[6, 5, 4, 4, 4, 3, 3, 2, 2, 2, 1], [11, 8, 7, 5, 4, 1]]
[5, 4, 3, 2, 2] [6, 4, 4, 3, 2, 1]
[6, 4, 3, 2, 2, 1] [5, 4, 4, 3, 2]
=====
[[6, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2], [10, 8, 6, 6, 4, 2]]
[4, 4, 2, 2, 2] [6, 4, 4, 4, 2, 2]
[6, 4, 2, 2, 2, 2] [4, 4, 4, 4, 2]
=====
[[6, 5, 4, 3, 3, 3, 3, 3, 3, 2, 1], [11, 7, 6, 6, 5, 1]]
[5, 3, 3, 3, 2] [6, 4, 3, 3, 3, 1]
[6, 3, 3, 3, 2, 1] [5, 4, 3, 3, 3]
=====
[[6, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2], [10, 7, 7, 5, 5, 2]]
[4, 3, 3, 2, 2] [6, 4, 4, 3, 3, 2]
[6, 3, 3, 2, 2, 2] [4, 4, 4, 3, 3]
=====
[[5, 5, 5, 4, 4, 3, 3, 2, 2, 1, 1, 1], [10, 9, 7, 5, 3, 

In [25]:
lrcalc.mult([2, 2], [3, 1, 1])
#dominance order min ([3,2,2,1,1], [5,3,1])

{[3, 3, 2, 1]: 1,
 [4, 2, 2, 1]: 1,
 [4, 3, 1, 1]: 1,
 [3, 2, 2, 1, 1]: 1,
 [4, 2, 1, 1, 1]: 1,
 [5, 2, 1, 1]: 1,
 [3, 3, 3]: 1,
 [4, 3, 2]: 1,
 [5, 3, 1]: 1}

In [21]:
lrcalc.mult([2, 1],[3, 2, 1])
#dominance order min ([3,2,2,1,1], [5,3,1])

{[3, 2, 2, 2]: 1,
 [3, 3, 2, 1]: 2,
 [4, 2, 2, 1]: 2,
 [4, 3, 1, 1]: 2,
 [3, 2, 2, 1, 1]: 1,
 [4, 2, 1, 1, 1]: 1,
 [5, 2, 1, 1]: 1,
 [3, 3, 1, 1, 1]: 1,
 [3, 3, 3]: 1,
 [4, 3, 2]: 2,
 [4, 4, 1]: 1,
 [5, 2, 2]: 1,
 [5, 3, 1]: 1}

In [62]:

for a in range(6,8):
    for b in range(6,8):

        print(a,b)
        lams = subsetPartRec(a,b)
#         lamsCPlus = [plusOne(e,a) for e in lamsComp(lams,a,b)]
        lamsC = lamsComp(lams,a,b)

        indexDict = []
        ind = 0

        vectors = []
#         vectorsPlus = []

        for i in range(len(lams)):
            vectors.append(schurVec(lams[i],lamsC[i]))
#             vectorsPlus.append(schurVec(lams[i],lamsCPlus[i]))
        
        A = span(vectors,ZZ)
        print(A.dimension() == len(vectors))
        

6 6
True
6 7
True
7 6


KeyboardInterrupt: 

In [132]:
lamsCPlus[i]
lamsC[0]

[2, 2]

In [137]:

for a in range(2,3):
    for b in range(2,3):
#         a = 4
#         b = 3
        print(a,b)
        lams = subsetPartRec(a,b)
        lamsCPlus = [plusOne(e,a) for e in lamsComp(lams,a,b)]
        lamsC = lamsComp(lams,a,b)
        

        indexDict = {}
        ind = 0

        vectors = []
        vectorsPlus = []

        for i in range(len(lams)):
            vectors.append(schurVec(lams[i],lamsC[i]))
            vectorsPlus.append(schurVec(lams[i],lamsCPlus[i]))
            
        maxInd = np.max(np.nonzero(vectorsPlus[-1]))

        first = []
        second = []
        
        print(lams[1],lamsC[1],lamsCPlus[1])

        for i in range(len(vectors)):
            v = vectors[i][:maxInd+1]
            print("====")
            first = []
            
            for u in (vectors[:i]+vectors[i+1:]):
                u = u[:maxInd+1]
                current = []
                for j in range(len(v)):
                    if v[j] > 0 and u[j] == 0:
                        current.append([k for k,v in indexDict.items() if v == j][0])
                first.append(current)
                if current == [[2,1,1]]:
                    print(i,v,u)
            
            second = []
            v = vectorsPlus[i][:maxInd+1]
#             print(v)
            for u in (vectorsPlus[:i]+vectorsPlus[i+1:]):
                u = u[:maxInd+1]
                current = []
                for j in range(len(v)):
                    if v[j] > 0 and u[j] == 0:
                        current.append([k for k,v in indexDict.items() if v == j][0])
                second.append(current)
                if current == []:
                    print(i,v,u)

            print(first)
            print(second)
            for i,e in enumerate(second):
                flag = 0
                for nu in first[i]:
                    current = [x+y for x,y in zip_longest(nu, [1]*a, fillvalue=0)]
                    if current in second[i]:
                        flag = 1
                if flag == 0 and len(first[i]) != 0:
                    print("!!!!!!!!!!!!!!", first[i])
        

2 2
[1] [2, 1] [3, 2]
====
0 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]
0 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0]
0 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[[], [], []]
[[], [], []]
====
1 [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
1 [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0]
[[[3, 1], [2, 1, 1]], [[2, 1, 1]], [[3, 1]]]
[[[4, 2], [3, 2, 1]], [], [[4, 2]]]
!!!!!!!!!!!!!! [[2, 1, 1]]
====
[[[3, 1], [4]], [[4]], [[3, 1], [4]]]
[[[4, 2], [3, 2, 1], [5, 1], [4, 1, 1]], [[5, 1], [4, 1, 1]], [[4, 2], [5, 1], [4, 1, 1]]]
====
[[[2, 1, 1], [1, 1, 1, 1]], [[1, 1, 1, 1]], [[2, 1, 1], [1, 1, 1, 1]]]
[[[3, 2, 1], [2, 2, 1, 1]], [[2, 2, 1, 1]], [[2, 2, 1, 1]]]


In [122]:
indexDict

{[2, 2]: 1,
 [3, 3]: 2,
 [3, 1]: 3,
 [2, 1, 1]: 4,
 [4, 2]: 5,
 [3, 2, 1]: 6,
 [4]: 7,
 [5, 1]: 8,
 [4, 1, 1]: 9,
 [1, 1, 1, 1]: 10,
 [2, 2, 1, 1]: 11}

In [101]:
maxInd = 0
for v in vectorsPlus:
    current = np.max(np.nonzero(v))
    if maxInd < current: 
        maxInd = current
        
first = []

for i in range(len(vectors)-1):
    v = vectors[i][:maxInd+1]
#     print(lams[i], lamsC[i], "\n", v)
#     print(v)
    current = []
    for j in range(len(v)):
        if v[j] > 0 and vectors[i+1][j] == 0:
            current.append([k for k,v in indexDict.items() if v == j][0])
#             print(j, [k for k,v in indexDict.items() if v == j])
    first.append(current)

print("\n\n")

second = []

for i in range(len(vectors)-1):
    v = vectorsPlus[i][:maxInd+1]
    print(lams[i], lamsCPlus[i], "\n", v)
#     print(v)
    current = []
    for j in range(len(v)):
        if v[j] > 0 and vectorsPlus[i+1][j] == 0:
            current.append([k for k,v in indexDict.items() if v == j][0])
    second.append(current)
            
# for i,v in enumerate(vectors):
#     print(lams[i], lamsCPlus[i], "\n", vectorsPlus[i][:maxInd+1])
#     print(lams[i], lamsC[i])


In [107]:
for i,e in enumerate(second):
    flag = 0
    for nu in first[i]:
        current = [x+y for x,y in zip_longest(nu, [1]*a, fillvalue=0)]
        if current in second[i]:
            flag = 1
    if flag == 0 and len(first[i]) != 0:
        print(first[i])

In [32]:
print(first,"\n\n\n",second)

[[], [], [[4, 3, 1, 1], [5, 3, 1]], [[3, 3, 2, 1], [4, 2, 2, 1], [3, 2, 2, 1, 1]], [[6, 3]], [[4, 3, 2], [4, 3, 1, 1], [5, 3, 1], [4, 2, 2, 1], [3, 2, 2, 2], [4, 2, 1, 1, 1], [5, 2, 1, 1], [3, 3, 1, 1, 1], [4, 4, 1], [5, 2, 2]], [[3, 2, 2, 1, 1], [2, 2, 2, 1, 1, 1]], [[6, 3], [4, 4, 1], [5, 2, 2], [5, 4], [6, 2, 1]], [[5, 3, 1], [5, 2, 1, 1]]] 


 [[], [], [[4, 4, 2, 2], [5, 4, 2, 1], [6, 4, 2]], [[5, 3, 3, 1], [4, 3, 3, 1, 1]], [[7, 4, 1], [6, 4, 1, 1]], [[5, 4, 3], [4, 4, 2, 2], [5, 4, 2, 1], [6, 4, 2], [5, 3, 3, 1], [4, 4, 2, 1, 1], [4, 3, 3, 2], [5, 3, 2, 2], [4, 3, 2, 2, 1], [6, 3, 2, 1], [5, 3, 2, 1, 1], [5, 5, 2], [6, 3, 3]], [[3, 3, 3, 1, 1, 1]], [[7, 4, 1], [6, 4, 1, 1], [4, 4, 2, 1, 1], [4, 3, 3, 2], [5, 5, 2], [6, 3, 3], [7, 3, 2], [7, 3, 1, 1], [5, 4, 1, 1, 1], [5, 5, 1, 1], [6, 3, 1, 1, 1], [6, 5, 1]], [[6, 4, 2], [5, 3, 2, 2], [6, 3, 2, 1], [4, 2, 2, 2, 2], [5, 2, 2, 2, 1], [6, 2, 2, 2]]]


In [ ]:
maxInd = 0
for v in vectorsPlus:
    current = np.max(np.nonzero(v))
    if maxInd < current: 
        maxInd = current
for i,v in enumerate(vectors):
    print(lams[i], lamsC[i], "\n", v[:maxInd+1])
print("\n\n")
for i,v in enumerate(vectors):
    print(lams[i], lamsCPlus[i], "\n", vectorsPlus[i][:maxInd+1])
#     print(lams[i], lamsC[i])



In [13]:
maxInd = 0
for v in vectors:
    current = np.max(np.nonzero(v))
    if maxInd < current: 
        maxInd = current
for i,v in enumerate(vectors):
    print(lams[i], lamsC[i], "\n", v[:maxInd+1])
#     print(lams[i], lamsC[i])


[] [3, 3, 3] 
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1] [3, 3, 2] 
 [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2] [3, 3, 1] 
 [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1] [3, 2, 2] 
 [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3] [3, 3] 
 [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 1] [3, 2, 1] 
 [0, 1, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1] [2, 2, 2] 
 [0, 1, 0, 

In [14]:
indexDict

{[3, 3, 3]: 1,
 [4, 4, 4]: 2,
 [3, 3, 2, 1]: 3,
 [4, 3, 2]: 4,
 [4, 4, 3, 1]: 5,
 [5, 4, 3]: 6,
 [4, 3, 1, 1]: 7,
 [5, 3, 1]: 8,
 [4, 4, 2, 2]: 9,
 [5, 4, 2, 1]: 10,
 [6, 4, 2]: 11,
 [4, 2, 2, 1]: 12,
 [3, 2, 2, 1, 1]: 13,
 [5, 3, 3, 1]: 14,
 [4, 3, 3, 1, 1]: 15,
 [6, 3]: 16,
 [7, 4, 1]: 17,
 [6, 4, 1, 1]: 18,
 [3, 2, 2, 2]: 19,
 [4, 2, 1, 1, 1]: 20,
 [5, 2, 1, 1]: 21,
 [3, 3, 1, 1, 1]: 22,
 [4, 4, 1]: 23,
 [5, 2, 2]: 24,
 [4, 4, 2, 1, 1]: 25,
 [4, 3, 3, 2]: 26,
 [5, 3, 2, 2]: 27,
 [4, 3, 2, 2, 1]: 28,
 [6, 3, 2, 1]: 29,
 [5, 3, 2, 1, 1]: 30,
 [5, 5, 2]: 31,
 [6, 3, 3]: 32,
 [2, 2, 2, 1, 1, 1]: 33,
 [3, 3, 3, 1, 1, 1]: 34,
 [5, 4]: 35,
 [6, 2, 1]: 36,
 [7, 3, 2]: 37,
 [7, 3, 1, 1]: 38,
 [5, 4, 1, 1, 1]: 39,
 [5, 5, 1, 1]: 40,
 [6, 3, 1, 1, 1]: 41,
 [6, 5, 1]: 42,
 [4, 2, 2, 2, 2]: 43,
 [5, 2, 2, 2, 1]: 44,
 [6, 2, 2, 2]: 45,
 [2, 2, 2, 2, 1]: 46,
 [3, 2, 1, 1, 1, 1]: 47,
 [3, 3, 2, 2, 1, 1]: 48,
 [4, 3, 2, 1, 1, 1]: 49,
 [3, 3, 3, 2, 1]: 50}

In [ ]:
for p in indexDict:
    if not Partition(p).dominates(Partition([3, 3, 3, 1, 1, 1])):
        print(p)

In [178]:
first = Partition([10,5]).dominated_partitions()
second = Partition([3, 3, 3, 2, 2, 2]).dominated_partitions()
res = [p for p in first if p not in second]
res.append(Partition([3, 3, 3, 2, 2, 2]))
res = [p for p in res if len(p) <= 6]
res = [p for p in res if p not in indexDict]
len(res)

16

In [171]:
len(indexDict)

82

In [121]:
# given a rectangle a*b, an array of partitions \lambda_k
# determine whether s_\lambda_1 * s_\lambda_1_c .. are linearly independent

a = 2 #rows
b = 2 #cols

lams = [[2],[1,1],[2,1],[2,2]]
lamsC = lamsComp(lams,a,b)

indexDict = {}
ind = -1

vectors = []

for i in range(len(lams)):
    vectors.append(schurVec(lams[i],lamsC[i]))
    

A = span(vectors,ZZ)
A.dimension() == len(vectors)

True

In [122]:
Partitions(2, max_part=b, max_length=a).list()

[[2], [1, 1]]

In [140]:
basis = [[1,0,0],[0,1,0]]
F = ZZ
dim = len(basis[0])
VS = (F**dim).span_of_basis(basis)

In [206]:

for i in range(len(vectors)):
    if i == 0: 
        current = vectors[1:]
    else:
        current = vectors[:i] + vectors[i+1:]
    if span(current,ZZ).dimension() == 38:
        print(i)
        print(len(current))
        break


In [141]:
VS

Free module of degree 3 and rank 2 over Integer Ring
User basis matrix:
[1 0 0]
[0 1 0]

In [143]:
v = vector(F,[1,0,0])
c = VS.coordinate_vector(v)

In [144]:
c

(1, 0)